In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import NHiTSModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from pytorch_lightning.callbacks import Callback, EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [5]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

df = pd.read_csv("../data/ground_station_clean.csv")

In [6]:
encoders = {
    "cyclic":{
        "past":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()
}

In [7]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Criar diretório para salvar os resultados das iterações
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [8]:
def objective(trial):

    output_chunk_length = trial.suggest_int("output_chunk_length", 12, 72)
    input_chunk_length = trial.suggest_int("input_chunk_length", output_chunk_length + 1, output_chunk_length + 24 if output_chunk_length + 24 > output_chunk_length + 1 else output_chunk_length + 2) # Garante que o limite superior seja maior que o inferior
    num_layers    = trial.suggest_int("num_layers", 1, 6)
    num_blocks    = trial.suggest_int("num_blocks", 1, 4)
    num_stacks    = trial.suggest_int("num_stacks", 1, 5)
    layer_widths    = trial.suggest_int("layer_widths", 32, 256)
    activation = trial.suggest_categorical("activation", ["ReLU", "GELU"])
    dropout = trial.suggest_float('dropout', 0.0, 0.3)
    batch_size = trial.suggest_int("batch_size", 16, 64)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    forecast_horizon = trial.suggest_int("forecast_horizon", 1, 24)

    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
            "accelerator": "auto",
            "callbacks": [early_stopper],
    }
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nIniciando Trial {trial.number}")
    print(f"Hiperparâmetros: {trial.params}")
    print("\nTreinando o modelo...")
    print(f"Train set: {len(train_fold)} samples") # Usar len() para TimeSeries
    print(f"Validation set: {len(val_fold)} samples") # Usar len() para TimeSeries
    
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Definir work_dir e model_name para consistência
    # force_reset=True irá limpar work_dir/model_name antes de cada treinamento da iteração.
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/" # Certifique-se que este caminho existe e é gravável
    _model_name = "model_optuna_temp" # Nome do modelo temporário para cada iteração

    # Criar o diretório _work_dir se não existir
    os.makedirs(_work_dir, exist_ok=True)

    model = NHiTSModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        n_epochs=30, # Considere tornar n_epochs um hiperparâmetro ou usar um valor maior com EarlyStopping
        batch_size=batch_size,
        num_layers=num_layers,
        num_blocks=num_blocks,
        num_stacks=num_stacks,
        layer_widths=layer_widths,
        activation=activation,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,    
        show_warnings=True,
        force_reset=True, # Importante para que cada iteração treine do zero com este model_name
        optimizer_kwargs={"lr": lr, "weight_decay": 1e-5},
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=True # Pode ser False para menos output durante a otimização do Optuna
    )
    
    try:
        loaded_model = NHiTSModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Modelo carregado do checkpoint para trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint não encontrado para {_model_name} em {_work_dir}. Usando o modelo treinado em memória.")
        loaded_model = model # Usar o modelo treinado se o carregamento falhar (fallback)


    forecasts = loaded_model.historical_forecasts(
        train_scaled,# Como no script original
        start=0.8, # Início da previsão no conjunto de treinamento
        forecast_horizon=forecast_horizon,
        stride=1, # Gera uma previsão em cada ponto de tempo possível
        retrain=False,
        verbose=True # Pode ser False
    ) 

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]
    metrics = {}
    print("Iniciando a verificação das séries temporais para constância...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nMétricas de performance:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_smape_val = smape(s, forecasts_t)
    print(f"O SMAPE desse fold foi {overall_smape_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "num_layers": num_layers,
        "num_blocks": num_blocks,
        "num_stacks": num_stacks,
        "layer_widths": layer_widths,
        "activation": activation,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "forecast_horizon": forecast_horizon,
        "overall_smape_val": overall_smape_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Resultados do trial {trial.number} salvos em {json_path}")
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")



In [9]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [10]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-14 12:31:58,952] A new study created in memory with name: no-name-90094d3c-02da-4491-96f7-208f0de60621



Iniciando Trial 0
Hiperparâmetros: {'output_chunk_length': 51, 'input_chunk_length': 67, 'num_layers': 4, 'num_blocks': 4, 'num_stacks': 1, 'layer_widths': 178, 'activation': 'GELU', 'dropout': 0.10789878616487389, 'batch_size': 46, 'lr': 0.00047381332174476837, 'forecast_horizon': 4}

Treinando o modelo...
Train set: 6647 samples
Validation set: 1662 samples


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | HuberLoss        | 0      | train
1 | train_criterion | HuberLoss        | 0      | train
2 | val_criterion   | HuberLoss        | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 469 K  | train
-------------------------------------------------------------
460 K     Trainable params
8.6 K     Non-trainable params
469 K     Total params
1.878     Total estimated model params size (MB)
64        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

Modelo carregado do checkpoint para trial 49


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-14 13:00:43,355] Trial 49 finished with value: 55.74250564071785 and parameters: {'output_chunk_length': 30, 'input_chunk_length': 45, 'num_layers': 2, 'num_blocks': 4, 'num_stacks': 2, 'layer_widths': 146, 'activation': 'ReLU', 'dropout': 0.18980770884810852, 'batch_size': 37, 'lr': 0.009918862223348503, 'forecast_horizon': 11}. Best is trial 3 with value: 50.43235393618171.


Iniciando a verificação das séries temporais para constância...

Métricas de performance:
                                     MSE        RMSE         MAE          R2  \
Temperature                     2.855202    1.689734    1.314869    0.158680   
Precipitation_accumulated     425.285489   20.622451   17.859720 -446.424718   
Humidity                       27.592874    5.252892    4.527970    0.576131   
Wind_Speed_kmh                  0.301958    0.549507    0.407463    0.118260   
Soil_Moisture              249495.182682  499.494928  354.594316    0.664836   
Soil_Temperature                2.554671    1.598334    1.167509    0.543528   
Wind_Dir_Sin                    0.147993    0.384699    0.258313    0.084081   
Wind_Dir_Cos                    0.145760    0.381785    0.246919    0.494851   

                                SMAPE  
Temperature                 21.039876  
Precipitation_accumulated    3.802503  
Humidity                     5.413970  
Wind_Speed_kmh             17

In [11]:
print(f"Melhor Valor (Mínimo): {study.best_value}")
print(f"Melhores Parâmetros: {study.best_params}")

Melhor Valor (Mínimo): 50.43235393618171
Melhores Parâmetros: {'output_chunk_length': 27, 'input_chunk_length': 38, 'num_layers': 1, 'num_blocks': 3, 'num_stacks': 2, 'layer_widths': 85, 'activation': 'GELU', 'dropout': 0.18882539432465545, 'batch_size': 37, 'lr': 0.006016691509040486, 'forecast_horizon': 12}


In [12]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Melhores resultados salvos em {json_path}")

Melhores resultados salvos em optuna_iteration_metrics/best_trial.json
